In [3]:
pip install xlrd


  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [11]:
pip install openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import xlrd
import openpyxl
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# as the dataset has more than 2 million of rows so we have pre-processed the dataset via Excel (poor computer performance 
# to handle such large file in python). We only keep data recorded in the year of 2018, all columns that have more than half
# of the rows are 'NA' and all irrelevant columns which have no contribution to whether a loan should be given are removed.
data = pd.read_excel('../loan_2018_preprocessed.xlsx',engine='openpyxl')

data.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies
0,2500,36 months,13.56,84.92,10+ years,RENT,55000.0,Not Verified,Current,debt_consolidation,18.24,0,1,9,1,4341,10.3,34,1
1,30000,60 months,18.94,777.23,10+ years,MORTGAGE,90000.0,Source Verified,Current,debt_consolidation,26.52,0,0,13,1,12315,24.2,44,1
2,5000,36 months,17.97,180.69,6 years,MORTGAGE,59280.0,Source Verified,Current,debt_consolidation,10.51,0,0,8,0,4599,19.1,13,0
3,4000,36 months,18.94,146.51,10+ years,MORTGAGE,92000.0,Source Verified,Current,debt_consolidation,16.74,0,0,10,0,5468,78.1,13,0
4,30000,60 months,16.14,731.78,10+ years,MORTGAGE,57250.0,Not Verified,Current,debt_consolidation,26.35,0,0,12,0,829,3.6,26,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495242 entries, 0 to 495241
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             495242 non-null  int64  
 1   term                  495242 non-null  object 
 2   int_rate              495242 non-null  float64
 3   installment           495242 non-null  float64
 4   emp_length            453255 non-null  object 
 5   home_ownership        495242 non-null  object 
 6   annual_inc            495242 non-null  float64
 7   verification_status   495242 non-null  object 
 8   loan_status           495242 non-null  object 
 9   purpose               495242 non-null  object 
 10  dti                   494110 non-null  float64
 11  delinq_2yrs           495242 non-null  int64  
 12  inq_last_6mths        495242 non-null  int64  
 13  open_acc              495242 non-null  int64  
 14  pub_rec               495242 non-null  int64  
 15  

In [8]:
# check if any columns has only 1 value, such columns will not conribute to a classification model.
for col in data.columns:
    col_val = data[col].dropna().unique()
    print(col,len(col_val))

loan_amnt 1559
term 2
int_rate 110
installment 35297
emp_length 11
home_ownership 4
annual_inc 30071
verification_status 3
loan_status 7
purpose 13
dti 9464
delinq_2yrs 26
inq_last_6mths 6
open_acc 76
pub_rec 16
revol_bal 64413
revol_util 1136
total_acc 130
pub_rec_bankruptcies 8


In [9]:
# check the missing values. As the dataset still has about 500K of rows for the missing rows in a column is less 
# than 10K then it will be removed, otherwise we will fill the missing value.
null_count = data.isnull().sum()
print(null_count)

loan_amnt                   0
term                        0
int_rate                    0
installment                 0
emp_length              41987
home_ownership              0
annual_inc                  0
verification_status         0
loan_status                 0
purpose                     0
dti                      1132
delinq_2yrs                 0
inq_last_6mths              0
open_acc                    0
pub_rec                     0
revol_bal                   0
revol_util                592
total_acc                   0
pub_rec_bankruptcies        0
dtype: int64


In [20]:
# as the value of '10+ years' appereared the most so we will fill in using this value.
data.emp_length.unique()

array(['10+ years', '6 years', '4 years', '< 1 year', '2 years',
       '9 years', nan, '5 years', '3 years', '7 years', '1 year',
       '8 years'], dtype=object)

In [14]:
# the 'nan' or 'n/a' values will be filled by using the value which has occured the most below.
data.emp_length.value_counts()

10+ years    160382
2 years       44747
< 1 year      43627
3 years       40629
1 year        33167
4 years       31262
5 years       30854
6 years       21977
7 years       17961
8 years       16027
9 years       12622
Name: emp_length, dtype: int64

In [22]:
d1 = data.fillna({'emp_length':data['emp_length'].mode()[0]})
d1 = d1.dropna(axis=0)

d1.emp_length.unique()

array(['10+ years', '6 years', '< 1 year', '2 years', '4 years',
       '9 years', '5 years', '3 years', '7 years', '1 year', '8 years'],
      dtype=object)

In [23]:
d1.isnull().sum()

loan_amnt               0
term                    0
int_rate                0
installment             0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
loan_status             0
purpose                 0
dti                     0
delinq_2yrs             0
inq_last_6mths          0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
pub_rec_bankruptcies    0
dtype: int64

In [18]:
d1.dtypes.value_counts()

int64      8
object     6
float64    5
dtype: int64

In [19]:
object_col = d1.select_dtypes(include=['object'])

object_col.iloc[0]

term                            36 months
emp_length                      10+ years
home_ownership                       RENT
verification_status          Not Verified
loan_status                       Current
purpose                debt_consolidation
Name: 0, dtype: object

In [24]:
convert_emp_length_dict = {
    'emp_length':{
        '10+ years':10,
        '9 years':9,
        '8 years':8,
        '7 years':7,
        '6 years':6,
        '5 years':5,
        '4 years':4,
        '3 years':3,
        '2 years':2,
        '1 year' :1,
        '< 1 year':0
    }
}

d1 = d1.replace(convert_emp_length_dict)

d1.emp_length.unique()

array([10,  6,  0,  2,  4,  9,  5,  3,  7,  1,  8], dtype=int64)

In [26]:
# loan_status is our output value and we only interest in whether a loan is fully paid (meaning approval) or 
# charged off (meaning default), any other value will be ignored.

d1.loan_status.value_counts()

Current               435814
Fully Paid             40093
Charged Off             6914
Late (31-120 days)      6483
In Grace Period         2893
Late (16-30 days)       1314
Default                    9
Name: loan_status, dtype: int64

In [33]:
d2 = d1.loc[(d1['loan_status']=='Fully Paid')|(d1['loan_status']=='Charged Off')]

convert_loan_status_dict = {
    'loan_status':{
        'Fully Paid':1,
        'Charged Off':0
    }
}

d2 = d2.replace(convert_loan_status_dict)

d2.loan_status.unique()

array([1, 0], dtype=int64)

In [38]:
from sklearn.preprocessing import LabelEncoder

In [45]:
lbe = LabelEncoder()

cat_columns = ['term','home_ownership','verification_status','purpose']

d3 = d2[cat_columns]

for col in d3.columns:
    val = d3[col].unique().tolist()
    lbe.fit(val)
    d3[col]=lbe.transform(d3[col])
    print(f"{val},{lbe.fit_transform(val)}")

[' 36 months', ' 60 months'],[0 1]
['MORTGAGE', 'RENT', 'OWN', 'ANY'],[1 3 2 0]
['Source Verified', 'Verified', 'Not Verified'],[1 2 0]
['debt_consolidation', 'credit_card', 'other', 'house', 'car', 'home_improvement', 'moving', 'small_business', 'vacation', 'medical', 'major_purchase', 'renewable_energy', 'wedding'],[ 2  1  8  4  0  3  7 10 11  6  5  9 12]


C:\Users\Matt\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [54]:
d4 = d2.drop(cat_columns,axis=1)

d4 = pd.concat([d4,d3],axis=1)

d4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47007 entries, 100 to 495241
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   loan_amnt             47007 non-null  int64  
 1   int_rate              47007 non-null  float64
 2   installment           47007 non-null  float64
 3   emp_length            47007 non-null  int64  
 4   annual_inc            47007 non-null  float64
 5   loan_status           47007 non-null  int64  
 6   dti                   47007 non-null  float64
 7   delinq_2yrs           47007 non-null  int64  
 8   inq_last_6mths        47007 non-null  int64  
 9   open_acc              47007 non-null  int64  
 10  pub_rec               47007 non-null  int64  
 11  revol_bal             47007 non-null  int64  
 12  revol_util            47007 non-null  float64
 13  total_acc             47007 non-null  int64  
 14  pub_rec_bankruptcies  47007 non-null  int64  
 15  term            

In [55]:
d4.duplicated().sum()

0

In [56]:
d4.reset_index(drop=True)

d4.to_csv('cleaned_loan.csv',index=False)